In [ ]:
import pymysql.cursors

connection = pymysql.connect(host='',
                             user='',
                             password='',
                             db='temp',
                             cursorclass=pymysql.cursors.DictCursor)
cursor = connection.cursor()

file = "../million.txt"
blocksize = 100
table = "million"

In [ ]:
with open(file) as f:
    line = f.readline().strip()
    while line:
        values = []
        for _ in range(blocksize):
            if not line:
                print("Finished")
                break
            station, temperature = line.split(";")
            values.append(f'("{station}", {temperature})')
            line = f.readline().strip()
        if values:
            sql = f"INSERT INTO {table} (station, temperature) VALUES {', '.join(values)}"
            # print(sql)
            cursor.execute(sql)

print("Committed")
connection.commit()

# writing the million rows over the network to the pi takes: 7 minutes (blocksize = 10)
# writing directly at the raspberry pi takes: 55s (blocksize = 10), 20s (blocksize=100)

In [ ]:
cursor.execute(f"select count(*) from {table}")
cursor.fetchall()

In [ ]:
sql = f"SELECT station, min(temperature) as min, avg(temperature) as mean, max(temperature) as max from {table} group by station order by station"
print(sql)

In [ ]:
# calculating the million on the pi using mysql takes 11s
cursor.execute(sql)
ret = cursor.fetchall()

In [ ]:
for row in ret:
    print(f"{row['station']}={row['min']:0.1f};{row['mean']:0.1f};{row['max']:0.1f}")

In [ ]:
connection.close()

In [ ]:
cursor.execute(f"delete from {table}")
connection.commit()